In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
# !pip install scikit-learn
# !pip install albumentations
# !pip install opencv-python
# !pip install tqdm
# !pip install torchinfo
# !pip install segmentation-models-pytorch
# !pip install matplotlib
# !pip install scipy
# !pip install onnx
# !pip install mlflow
# !pip install onnxruntime-gpu

In [3]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from tqdm import tqdm
import math
from torchinfo import summary
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2 as cv
import copy
torch.manual_seed(3407)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="mobileone_s4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                     # model output channels (number of classes in your dataset)
)

In [5]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
model_student = model.to(device)#.train()
print(summary(model=model_student,
        input_size=(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]))

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Trainable
Unet (Unet)                                        [1, 3, 224, 224]     [1, 2, 224, 224]     --                   True
├─MobileOne (encoder)                              [1, 3, 224, 224]     [1, 3, 224, 224]     --                   True
│    └─MobileOneBlock (stage0)                     [1, 3, 224, 224]     [1, 64, 112, 112]    --                   True
│    │    └─Sequential (rbr_scale)                 [1, 3, 224, 224]     [1, 64, 112, 112]    320                  True
│    │    └─ModuleList (rbr_conv)                  --                   --                   1,856                True
│    │    └─Identity (se)                          [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
│    │    └─ReLU (activation)                      [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
│    └─Sequential (stage1)                     

In [6]:
for param in model_student.parameters():
    param.requires_grad = False
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
model_student = model.to(device).train()
print(summary(model=model_student,
        input_size=(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]))

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Trainable
Unet (Unet)                                        [1, 3, 224, 224]     [1, 2, 224, 224]     --                   False
├─MobileOne (encoder)                              [1, 3, 224, 224]     [1, 3, 224, 224]     --                   False
│    └─MobileOneBlock (stage0)                     [1, 3, 224, 224]     [1, 64, 112, 112]    --                   False
│    │    └─Sequential (rbr_scale)                 [1, 3, 224, 224]     [1, 64, 112, 112]    (320)                False
│    │    └─ModuleList (rbr_conv)                  --                   --                   (1,856)              False
│    │    └─Identity (se)                          [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
│    │    └─ReLU (activation)                      [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
│    └─Sequential (stage1)                

In [7]:
for name, param in model_student.named_parameters():
    if "decoder" in name or "segmentation_head" in name:
        param.requires_grad = True
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
model_student = model.to(device).train()
print(summary(model=model_student,
        input_size=(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]))

Layer (type (var_name))                            Input Shape          Output Shape         Param #              Trainable
Unet (Unet)                                        [1, 3, 224, 224]     [1, 2, 224, 224]     --                   Partial
├─MobileOne (encoder)                              [1, 3, 224, 224]     [1, 3, 224, 224]     --                   False
│    └─MobileOneBlock (stage0)                     [1, 3, 224, 224]     [1, 64, 112, 112]    --                   False
│    │    └─Sequential (rbr_scale)                 [1, 3, 224, 224]     [1, 64, 112, 112]    (320)                False
│    │    └─ModuleList (rbr_conv)                  --                   --                   (1,856)              False
│    │    └─Identity (se)                          [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
│    │    └─ReLU (activation)                      [1, 64, 112, 112]    [1, 64, 112, 112]    --                   --
│    └─Sequential (stage1)              

In [8]:
train_transform = A.Compose([A.Resize(height = 224, width = 224),
                             A.HorizontalFlip(p=0.5),
                             # A.VerticalFlip(p=0.5),
                             # A.RandomRotate90(p=0.5),
                             A.Rotate(limit = (-40,40), p=0.4),
                             # A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
                             # A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.5),
                             # A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
                             A.augmentations.transforms.ColorJitter(brightness=0.6, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.6),
                             # A.RandomResizedCrop(width=224,height=224,p=0.3,scale=(0.2, 0.99)),
                             A.augmentations.geometric.transforms.Affine(scale=(0.7, 3.0),translate_percent=60,p=0.5),
                             # A.augmentations.blur.transforms.ZoomBlur(always_apply=False, p=0.2),
                             A.augmentations.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                             ToTensorV2()],is_check_shapes=False)

test_transforms = A.Compose([A.Resize(height=224, width=224),
                             A.augmentations.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                            ToTensorV2()],is_check_shapes=False)

In [9]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

batch_size = 8
num_workers = 0

class GetDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir) # list all tye files in directory(folder)

    def __len__(self): # check length of that dataset
        return len(self.images)

    def __getitem__(self, index): # take as input some index
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".bmp", ".png").replace(".jpeg",".png")).replace(".jpg", ".png")
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        label = mask/255.0
        if self.transform is not None:
            augmentations_ori = self.transform(image=image, mask=label) # pass an image that want to augment
            image = augmentations_ori["image"]
            mask = augmentations_ori["mask"]


        return image, mask

image_train_dir = r"Train\img"
image_train_mask_dir = r"Train\mask"
image_test_dir = r"Test\img"
image_test_mask_dir = r"Test\mask"
train_dataset = GetDataset(image_train_dir,image_train_mask_dir,transform = train_transform)
test_dataset = GetDataset(image_test_dir,image_test_mask_dir,transform = test_transforms)

print(f"train_dataset {len(train_dataset)}")
print(f"test_dataset {len(test_dataset)}")

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers = num_workers,shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, num_workers = num_workers,shuffle=True, pin_memory=True)

train_dataset 2282
test_dataset 385


In [10]:
def cal_loss(pred, target, classes=2):
    ce_loss = torch.nn.functional.cross_entropy(pred, target, reduction='none') # important to add reduction='none' to keep per-batch-item loss
    pt = torch.exp(-ce_loss)
    alpha = 0.25 # default 0.25
    gamma = 2 # default 2
    focal_loss = (alpha * (1-pt)**gamma * ce_loss).mean() # mean over the batch

    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(pred),axis=1)

    dice_score_all = []
    for i in range(classes):
        preds_dice = copy.deepcopy(preds)
        preds_dice[preds_dice != i] = 0
        target_dice = copy.deepcopy(target)
        target_dice[target_dice != i] = 0
        dice_score_all.append(dice_coef(target_dice,preds_dice))

    dice_score = np.array(dice_score_all).mean()
    dice_loss = torch.from_numpy(np.asarray(1 - dice_score))

    scale_focal = 10 # default 10
    scale_dice = 1

    loss_hard = scale_focal * focal_loss + scale_dice * (dice_loss)
    return loss_hard

def dice_coef(y_true, y_pred, epsilon=1e-8):
    y_true_flatten = np.asarray(y_true.to('cpu'), dtype=np.int64).astype(np.bool_)
    y_pred_flatten = np.asarray(y_pred.to('cpu'), dtype=np.int64).astype(np.bool_)

    if not np.sum(y_true_flatten) + np.sum(y_pred_flatten):
        return 1.0

    return (2. * np.sum(y_true_flatten * y_pred_flatten)) /\
        (np.sum(y_true_flatten) + np.sum(y_pred_flatten) + epsilon)

In [12]:
import copy
import mlflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment('human-segment') 
mlflow.pytorch.autolog()
epochs = 300
optimizer_student = torch.optim.Adam(model_student.parameters(), lr=1e-3, weight_decay=1e-4)
min_test_loss = float('inf')
min_train_loss = float('inf')
with mlflow.start_run():
    mlflow.log_param("lr", 1e-3)
    mlflow.log_param("l2", 1e-4)
    for epoch in range(epochs):
        loop_train = tqdm(enumerate(train_loader),total=len(train_loader))
        model_student.train()
        loss_overall_train = []
        for batch_idx, (image, label) in loop_train:
            image = image.float().to(device)
            label = label.to(device).long()
            optimizer_student.zero_grad()
            student_predict = model_student(image)
            loss_all = cal_loss(student_predict, label)
            loss_all.backward()
            optimizer_student.step()
            loss_overall_train.append(loss_all.detach().item())
    
        loss_overall_test = []
        loop_test = tqdm(enumerate(test_loader),total=len(test_loader))
        model_student.eval()
        for batch_idx, (image, label) in loop_test:
            image = image.float().to(device)
            label = label.to(device).long()
            with torch.no_grad():
                student_predict = model_student(image)
                loss_all = cal_loss(student_predict, label)
            loss_overall_test.append(loss_all.detach().item())
    
            softmax = nn.Softmax(dim=1)
            preds_student = torch.argmax(softmax(student_predict),axis=1)
        mlflow.log_metric("train_loss", np.mean(loss_overall_train), step=epoch)
        mlflow.log_metric("val_loss", np.mean(loss_overall_test), step=epoch)
        if np.mean(loss_overall_test) < min_test_loss and np.mean(loss_overall_train) < min_train_loss:
            print("save_model")
            example_forward_input_for_int8 = torch.rand(1, 3, 224, 224).to('cuda')
            for param in model_student.parameters():
                param.requires_grad = False
            model_student_to_trace = model_student.to('cuda').eval()
            onnx_program = torch.onnx.export(model_student_to_trace, example_forward_input_for_int8, r"Model\human_segment_{}.onnx".format(epoch))
    
    
    
            min_test_loss = np.mean(loss_overall_test)
            min_train_loss = np.mean(loss_overall_train)
            model_student.to(device)
            for name, param in model_student.named_parameters():
                if "decoder" in name or "segmentation_head" in name:
                    param.requires_grad = True

            f, axarr = plt.subplots(1,3, figsize=(200, 200))
            axarr[0].imshow(image[-1,0,:,:].cpu().detach().numpy(), cmap="gray")
            axarr[1].imshow(preds_student[-1].cpu().detach().numpy())
            axarr[2].imshow(label[-1].cpu().detach().numpy())
            plt.show()
    
        print()
        print(f"epoch {epoch+1}")
        print(f"loss_overall_train {np.mean(loss_overall_train)}")
        print(f"loss_overall_test {np.mean(loss_overall_test)}")

2024/03/03 16:29:30 INFO mlflow.tracking.fluent: Experiment with name 'human-segment' does not exist. Creating a new experiment.
2024/03/03 16:29:30 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
100%|██████████| 49/49 [00:17<00:00,  2.82it/s]
C:\Users\suppa\.conda\envs\human-segment\lib\site-packages\segmentation_models_pytorch\base\model.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if h % output_stride != 0 or w % output_stride != 0:


save_model



epoch 1
loss_overall_train 0.24029228263313435
loss_overall_test 0.24014337942426198


100%|██████████| 49/49 [00:17<00:00,  2.80it/s]


save_model


...


epoch 58
loss_overall_train 0.11244570889841041
loss_overall_test 0.16472453400782128


100%|██████████| 49/49 [00:16<00:00,  2.92it/s]



epoch 59
loss_overall_train 0.10531607506523087
loss_overall_test 0.16308039434871255


 52%|█████▏    | 150/286 [01:24<01:17,  1.77it/s]


KeyboardInterrupt: 

In [49]:
import numpy as np
import onnxruntime
import os
from onnxruntime.quantization import CalibrationDataReader
from PIL import Image
from onnxruntime import quantization
import onnx

def _preprocess_images(images_folder: str, height: int, width: int, size_limit=0):
    """
    Loads a batch of images and preprocess them
    parameter images_folder: path to folder storing images
    parameter height: image height in pixels
    parameter width: image width in pixels
    parameter size_limit: number of images to load. Default is 0 which means all images are picked.
    return: list of matrices characterizing multiple images
    """
    image_names = os.listdir(images_folder)
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
    unconcatenated_batch_data = []

    for image_name in batch_filenames:
        image_filepath = images_folder + "/" + image_name
        pillow_img = Image.new("RGB", (width, height))
        pillow_img.paste(Image.open(image_filepath).resize((width, height)))
        input_data = np.float32(pillow_img)/255.0
        input_data -= (0.485, 0.456, 0.406)
        input_data /= (0.229, 0.224, 0.225)
        nhwc_data = np.expand_dims(input_data, axis=0)
        nchw_data = nhwc_data.transpose(0, 3, 1, 2)  # ONNX Runtime standard
        unconcatenated_batch_data.append(nchw_data)
    batch_data = np.concatenate(
        np.expand_dims(unconcatenated_batch_data, axis=0), axis=0
    )
    return batch_data


class Human_segment_DataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder: str, model_path: str):
        self.enum_data = None

        # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, None)
        (_, _, height, width) = session.get_inputs()[0].shape

        # Convert image to input data
        self.nhwc_data_list = _preprocess_images(
            calibration_image_folder, height, width, size_limit=0
        )
        self.input_name = session.get_inputs()[0].name
        self.datasize = len(self.nhwc_data_list)

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(
                [{self.input_name: nhwc_data} for nhwc_data in self.nhwc_data_list]
            )
        return next(self.enum_data, None)

    def rewind(self):
        self.enum_data = None

print("start preprocess")
model_fp32_path = r'human_segment_29.onnx'
model_prep_path = r'human_segment_29_pre.onnx'
quantization.shape_inference.quant_pre_process(model_fp32_path, model_prep_path, skip_symbolic_shape=False)

print("read data")
input_model_path = r'human_segment_29_pre.onnx'
calibration_dataset_path = r"Caribrate"
dr = Human_segment_DataReader(
        calibration_dataset_path, input_model_path
    )

q_static_opts = {"ActivationSymmetric":False,
                 "WeightSymmetric":True}
if torch.cuda.is_available():
  q_static_opts = {"ActivationSymmetric":True,
                  "WeightSymmetric":True}
print("quantize")
model_prep_path = r'human_segment_29_pre.onnx'
model_int8_path = r'human_segment_29_int8.onnx'
quantized_model = quantization.quantize_static(model_input=model_prep_path,
                                               model_output=model_int8_path,
                                               calibration_data_reader=dr,
                                               quant_format=quantization.QuantFormat.QDQ,
                                               weight_type=quantization.QuantType.QInt8,
                                               per_channel=True,
                                               reduce_range=True,
                                               extra_options=q_static_opts)
print("finished")

start preprocess
read data
quantize
finished
